In [56]:
import glob 
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import plotly.graph_objs as go
# import plotly.offline as py
# import plotly.tools as tls
# import seaborn as sns
# import scipy.io.wavfile
import scipy
import csv

ERROR! Session/line number was not unique in database. History logging moved to new session 31


In [ ]:
import glob 
import keras
import IPython.display as ipd
import librosa
import librosa.display

import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
import tensorflow as tf
py.init_notebook_mode(connected=True)

## Rest
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm

import sklearn

In [1]:
import tensorflow as tf

In [5]:
df=pd.read_csv('dataset.csv', index_col=0)

ERROR! Session/line number was not unique in database. History logging moved to new session 21


In [6]:
df.head()

,path,actor,gender,emotion_label,emotion
0,RAVDESS/Actor_01/03-01-01-01-01-01-01.wav,1,male,neutral,1
1,RAVDESS/Actor_01/03-01-01-01-01-02-01.wav,1,male,neutral,1
2,RAVDESS/Actor_01/03-01-01-01-02-01-01.wav,1,male,neutral,1
3,RAVDESS/Actor_01/03-01-01-01-02-02-01.wav,1,male,neutral,1
4,RAVDESS/Actor_01/03-01-02-01-01-01-01.wav,1,male,calm,2


In [52]:
max_audio_duration=3
sample_rate=48000
max_audio_len=max_audio_duration*sample_rate

In [90]:
for i in range(df.shape[0]):
    path=df.path[i]
    data,sr=librosa.load(path, sr=sr, res_type='kaiser_fast' )
    data,_ = librosa.effects.trim(data,top_db = 25)
    data = scipy.signal.wiener(data)

    if len(data) > max_audio_len:
        data = data[0:max_audio_len]
    elif  max_audio_len > len(data):
        max_offset = max_audio_len - len(data)  
        data = np.pad(data, (0, max_offset), "constant")
    mfccs = np.mean(librosa.feature.mfcc(data, sr=sample_rate, n_mfcc=128).T, axis=0)
    delta=librosa.feature.delta(mfccs, delta=1)
    delta2=librosa.feature.delta(mfccs, delta=2)
    file = open('mfccs.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(mfccs)
    file2 = open('delta.csv', 'a', newline='')
    with file2:
        writer = csv.writer(file2)
        writer.writerow(delta)
    file3 = open('delta2.csv', 'a', newline='')
    with file3:
        writer = csv.writer(file3)
        writer.writerow(delta2)

In [91]:
audio_duration = 3
sampling_rate = 22050*2
input_length = sampling_rate * audio_duration
n_mfcc = 20
data_sample= np.zeros(input_length)
mfcc = librosa.feature.mfcc(data_sample, sr=sampling_rate, n_mfcc=n_mfcc)

In [ ]:
audios= np.empty(shape=(df.shape[0],128, mfcc.shape[1], 1))

count=0
for i in range(len(df)):
    signal, sample_rate = librosa.load(df.path[i], res_type='kaiser_fast',sr=sampling_rate)
    signal,index = librosa.effects.trim(signal,top_db = 25)
    signal = scipy.signal.wiener(signal)
    
    if len(signal) > input_length:
        signal = signal[0:input_length]
    elif  input_length > len(signal):
        max_offset = input_length - len(signal)  
        signal = np.pad(signal, (0, max_offset), "constant")

    melspec = librosa.feature.melspectrogram(signal, sr=sample_rate, n_mels=128,n_fft=2048,hop_length=512)   
    logspec = librosa.amplitude_to_db(melspec)
    logspec = np.expand_dims(logspec, axis=-1)
    audios[count,] = logspec 
    count+=1

In [98]:
np.save('logspec', audios)